In [ ]:
class Classroom:
    def __init__(self, name, capacity, room_type):
        self.name = name
        self.capacity = capacity
        self.room_type = room_type
    def __str__(self):
        return f"Classroom {self.name} (Capacity: {self.capacity}, Type: {self.room_type})"

class Professor:
    def __init__(self, name, preferred_days=[]):
        self.name = name
        self.preferred_days = preferred_days

class Assistant:
    def __init__(self, name):
        self.name = name

class Timetable:
    def __init__(self,timetable,fitness_score):
        self.timetable = timetable
        self.fitness_score = fitness_score

classrooms = [
    Classroom("C1", capacity=30, room_type="Vorlesung"),
    Classroom("C2", capacity=25, room_type="Vorlesung"),
    Classroom("C3", capacity=20, room_type="Vorlesung"),
    Classroom("C4", capacity=20, room_type="Vorlesung"),
    Classroom("C5", capacity=15, room_type="Vorlesung"),
    Classroom("C6", capacity=15, room_type="Lab"),
    Classroom("C7", capacity=10, room_type="Lab")
]
professors = [
    Professor("Dr. Smith", preferred_days=["Monday", "Wednesday"]),
    Professor("Prof. Johnson", preferred_days=["Tuesday", "Thursday"]),
    Professor("Prof. Yıldız", preferred_days=["Tuesday", "Friday"])
]
timeslots = ["9-10", "10-11", "11-12", "12-13", "13-14", "14-15", "15-16", "16-17", "17-18"]
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

import random

class Lesson:
    def __init__(self, code, teacher, name, is_online, class_type, preferred_days=[], timeslot=None, duration=1, room_preference=[],capacity=None):
        self.code = code
        self.teacher = teacher
        self.name = name
        self.is_online = is_online
        self.class_type = class_type
        self.preferred_days = teacher.preferred_days
        self.timeslot = timeslot
        self.duration = duration
        self.room_preference = room_preference
        self.capacity = capacity

    def __str__(self):
        return self.code


In [ ]:
lessons = [
  Lesson(
    code="INF101",
    teacher=professors[0],  # Dr. Smith
    name="Introduction to Computer Science",
    is_online=False,
    class_type="Lecture",
    duration=2,
    room_preference=classrooms[:2],  # Preferable classrooms C1, C2, C3
    capacity=30  # Required capacity
  ),

  Lesson(
    code="INF104",
    teacher=professors[1],  # Prof. Johnson
    name="Data Structures and Algorithms",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference= [classrooms[0], classrooms[1], classrooms[2], classrooms[3]],
    capacity=25  # Required capacity
  ),

  Lesson(
    code="INF205",
    teacher=professors[2],  # Prof. Yıldız
    name="Introduction to Artificial Intelligence",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[3:5],
    capacity=20  # Required capacity
  ),

  Lesson(
    code="INF204",
    teacher=professors[0],  # Dr. Smith
    name="Database Management Systems",
    is_online=False,
    class_type="Vorlesung",
    duration=2,
    room_preference=classrooms[:3],
    capacity=30  # Required capacity
  ),

  Lesson(
    code="INF303",
    teacher=professors[1],  # Prof. Johnson
    name="Software Engineering",
    is_online=False,
    class_type="Lab",
    duration=2,
    room_preference=classrooms[4:6],
    capacity=25  # Required capacity
  )
]

In [ ]:
import random

def generate_timetable2(lessons, classrooms, timeslots, days_of_week):
    timetable = {day: {timeslot: [None] * len(classrooms) for timeslot in timeslots} for day in days_of_week}
    for lesson in lessons:
        preferred_days = lesson.preferred_days.copy()
        random.shuffle(preferred_days)  # Shuffle preferred days to randomize selection
        for preferred_day in preferred_days:
            available_timeslots = [timeslot for timeslot in timeslots if all(slot == [None] * len(classrooms) for slot in timetable[preferred_day].values())]
            if not available_timeslots:
                continue

            # Ensure enough consecutive available timeslots for the lesson duration
            consecutive_available = 0
            start_indices = []
            for i, timeslot in enumerate(available_timeslots):
                if consecutive_available == lesson.duration:
                    break
                if i == 0 or timeslot - available_timeslots[i - 1] == 1:
                    consecutive_available += 1
                    if consecutive_available == 1:
                        start_indices.append(i)
                else:
                    consecutive_available = 1

            if consecutive_available < lesson.duration:
                continue  # Not enough consecutive available slots for the lesson duration

            start_index = random.choice(start_indices)  # Choose a random start index from available start indices
            selected_timeslot = available_timeslots[start_index]

            room = random.choice(lesson.room_preference)

            for i in range(start_index, start_index + lesson.duration):
                timeslot = available_timeslots[i]
                classroom_index = classrooms.index(room)
                timetable[preferred_day][timeslot][classroom_index] = lesson

            break  # Break out of preferred day loop once lesson is scheduled

    return timetable


timetable2 = generate_timetable(lessons, classrooms, timeslots, days_of_week)

In [ ]:
for day in days_of_week:
    print(f"Timetable for {day}:")
    print("       ", end="")
    for classroom in classrooms:
        print(f"{classroom.name:<10}", end="  ")  # Adjust the width as needed
    print("")

    for timeslot in timeslots:
        print(f"{timeslot:<6}", end="")
        for classroom in classrooms:
            lesson = timetable2[day][timeslot][classrooms.index(classroom)]  # Get the lesson for the current classroom
            if lesson is None:
                print(" " * 10, end="  ")  # Adjust the width as needed
            else:
                print(f"{lesson.code:<6}", end=" " * (10 - len(lesson.code) + 2))  # Adjust the width as needed
        print("")
    print("")


Timetable for Monday:
       C1          C2          C3          C4          C5          C6          C7          
9-10                                                                                      
10-11                                                                                     
11-12                                                                                     
12-13                                                                                     
13-14                                                                                     
14-15                                                                                     
15-16                                                                                     
16-17                                                                                     
17-18                                                                                     

Timetable for Tuesday:
       C1          C2          C3          

In [ ]:
def calculate_fitness(timetable):
    fitness = 0
    for day in timetable:
        for timeslot in timetable[day]:
            teachers_in_timeslot = []
            for lesson in timetable[day][timeslot]:
                if lesson is not None:
                    if lesson.teacher in teachers_in_timeslot:
                        fitness += 1
                    else:
                        teachers_in_timeslot.append(lesson.teacher)
    return fitness

In [ ]:
def roulette_wheel_selection(population, fitness_scores, num_parents):

    total_fitness = sum(fitness_scores)

    probabilities = [score / total_fitness for score in fitness_scores]

    selected_parents = []
    for _ in range(num_parents):
        r = random.random()
        cumulative_probability = 0
        for i, probability in enumerate(probabilities):
            cumulative_probability += probability
            if r <= cumulative_probability:
                selected_parents.append(population[i])
                break

    return selected_parents